In [283]:
#设计一个单特征值输入的五层的全连接神经网络，激活函数使用ReLU函数，达到对于y=x^2+2x-3的拟合
import numpy as np
neuronNum=[0,3,3,3,3,1]
wAll = {}
wAll[0] = None
bAll = {}
bAll[0] = None
input = {}
input[0] = np.random.randn(1, 1)
input[1] = input[0]
for i in range(2, 6):
    input[i] = np.ones((3, 1))
outputAll = {}
outputAll[0] = input[0]
linearAll = {}
linearAll[0] = None  
dwAll = {}
dbAll = {}
dlinearAll = {}
dLineardwAll = {}
dLineardbAll = {}
dback = {}

In [239]:
# 初始化每层的参数
def initialize(layerNum, neuronNum, shape):
    # 因为使用了ReLU函数，所以每一层的参数只有线性函数计算时使用到的系数w和偏置b
    # 因为这里的输入层输入只有x一个特征，所以每个隐层神经元和输出层神经元处理后也只能得到一个输出值
    # 所以输入层、输出层和隐层所接收到的都是一个一维向量，只不过对于输入层来说，这个一维向量只有一个分量
    # 所以初始化参数时根据每一层的神经元数量与上一层输出的一维向量的维数就可以确定这一层需要初始化的参数的维数，使用numpy.random.randn初始化即可
    w = np.random.randn(neuronNum, shape)
    wAll[layerNum] = w
    b = np.random.randn(neuronNum, 1)
    bAll[layerNum] = b
    return w, b


# 计算线性函数
def linear(w, b, input):
    linear = np.dot(w, input)+b
    return linear


# 计算ReLU函数
def ReLU(intermedium):
    ReLU = np.maximum(intermedium, 0)
    return ReLU

In [240]:
# 计算损失函数-->目标为最小化这个函数
def origin():
    output = np.dot(input[1], input[1])+2*input[1]-3
    np.dot(input[1], input[1])+2*input[1]-3
    return output


def cost(outputNN):
    ori=origin()
    cost = np.sqrt(outputNN-origin())
    print(outputNN)
    print(ori)
    if outputNN>ori:
        dcost=2*(outputNN-ori)
    else:
        dcost=2*(ori-outputNN)*(-1)
    return cost,dcost

In [241]:
# 计算每一层的正向计算-->求线性函数与ReLU函数处理后得到的值
def forward(neuronNum):
    # 第一层
    # 初始化第一层的参数
    for i in range(1,5):
        w, b = initialize(i,neuronNum[i], input[i].shape[0])
        wAll[i]=w
        bAll[i]=b
        liout = linear(w, b, input[i])
        linearAll[i] = liout
        output = ReLU(liout)
        print(f"第{i+1}层的输出为{output}")
        outputAll[i] = output
        input[i+1]=output
    # 此时i为4
    w, b = initialize(i+1, neuronNum[i+1], input[i+1].shape[0])
    wAll[i+1]=w
    bAll[i+1]=b
    liout = linear(w, b, input[i+1])
    linearAll[i+1] = liout
    output = ReLU(liout)
    print(f"第{i+1}层的输出为{output}")
    outputAll[i+1] = output
    costNN,dcost=cost(output)
    print(f"此轮迭代的损失为{costNN}")
    return output,costNN,dcost

In [242]:
# 计算每一层的反向计算-->求梯度
def backward(dcost):
    # 第五层
    # 先对ReLU求梯度
    
    i=5
    dlinear = outputAll[i]
    dlinear[dlinear < 0] = 0
    dlinear[dlinear > 0] = 1
    dlinearAll[i] = dlinear
    dlinear=dlinear*dcost
    # 然后对线性求梯度
    dLineardw = input[i]
    dLineardwAll[i] = dLineardw
    dLineardb = np.ones((input[i].shape[0], 1))
    dLineardbAll[i] = dLineardb
    # 然后利用链式法则求梯度
    dw = dlinear*dLineardw
    db = dlinear*dLineardb
    dwAll[i] = dw
    dbAll[i] = db
    dx=wAll[i]
    dback[i]=dlinear*dx
    i-=1
    for j in range(i,0,-1):
        # 下一层反向到这一层的时候需要乘一个dfundamental，这是整个模型的最终输出对上一层的输入求得的梯度，维度是3*1
        dfundamention=dback[j+1]
        dlinear = outputAll[j]
        dlinear[dlinear<0]=0
        dlinear[dlinear>0]=1 
        # dlinear是该层的输出对ReLU进行求梯度，维度是3*1，和dfundamental作逐元素相乘
        dlinearAll[j]=dlinear*dfundamention
        # 然后对线性求梯度
        # dlineardw 是该层的输出对该层的w进行求梯度
        dLineardw=input[j]
        dLineardwAll[j]=dLineardw
        # dlineardb 是该层的输出对该层的b进行求梯度
        dLineardb=np.ones((input[j].shape[0],1))
        dLineardbAll[j]=dLineardb
        # 然后利用链式法则求梯度
        # dlineardw 是模型的最终输出对该层的b进行求梯度
        dw=dlinear*dLineardw
        # dlineardb 是模型的最终输出对该层的b进行求梯度
        db=dlinear*dLineardb
        # 将该层求得的梯度放入散列表中
        dwAll[j]=dw
        dbAll[j]=db
        # 算出最终模型输出对该层的x的梯度，作为下一层的dfundamental
        dback[j]=dlinear*wAll[j]
    return dwAll,dbAll

In [280]:
# 定义一个手写神经网络类
def NnInHand(input,epoches):
    for i in range(epoches):
        print(f"第{i+1}次迭代>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        # 正向传播
        outputNN,costNN,dcost = forward(neuronNum)
        print(f"给模型最终输出为{outputNN}")
        # 逆向传播
        dwAll,dbAll=backward(dcost)
        print("各层参数的梯度为")
        print(dwAll)
        print(dbAll)
        # 以最小化损失为目的，迭代更新参数
        for j in range(1,6):
            print(f"第{j}层参数更新")
            wAll[j]=wAll[j]-a*dwAll[j]
            bAll[j]=bAll[j]-a*dbAll[j]
    # 计算模型的准确率
    print(f"acc:{(1-abs(costNN/origin()))*100}%")
    # 计算模型的损失
    print(f"loss:{abs((costNN/origin())*100)}%")
    return None

In [284]:
if __name__ == "__main__":
    a=0.1
    epochs=50
    NnInHand(input, epochs)

第1次迭代>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
第2层的输出为[[0.65265707]
 [1.27717213]
 [0.        ]]
第3层的输出为[[0.52718176]
 [2.14713144]
 [0.        ]]
第4层的输出为[[0.        ]
 [0.        ]
 [2.42751647]]
第5层的输出为[[0.        ]
 [4.17754081]
 [0.44236134]]
第5层的输出为[[0.]]
[[0.]]
[[-3.96337637]]
此轮迭代的损失为[[1.99082304]]
给模型最终输出为[[0.]]
各层参数的梯度为
{5: array([[0.],
       [0.],
       [0.]]), 4: array([[0.        ],
       [0.        ],
       [2.42751647]]), 3: array([[0.],
       [0.],
       [0.]]), 2: array([[0.65265707],
       [1.27717213],
       [0.        ]]), 1: array([[-0.80862698],
       [-0.80862698],
       [-0.        ]])}
{5: array([[0.],
       [0.],
       [0.]]), 4: array([[0.],
       [1.],
       [1.]]), 3: array([[0.],
       [0.],
       [1.]]), 2: array([[1.],
       [1.],
       [0.]]), 1: array([[1.],
       [1.],
       [0.]])}
第1层参数更新
第2层参数更新
第3层参数更新
第4层参数更新
第5层参数更新
第2次迭代>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
第2层的输出为[[0.        ]
 [0.    